In [1]:
%load_ext autoreload

In [2]:
from cap_package import SpotipyCollectPub as scp
from cap_package import SpotipyCollect as sc
from cap_package import ReadTransform as rt
from datetime import datetime
import datetime as dt
import demoji
from dotenv import load_dotenv
from IPython.utils.text import columnize
import os
import pandas as pd
from pathlib import Path
import re
import spotipy

... OK (Got response in 0.26 seconds)
Writing emoji data to C:\Users\Administrator\.demoji\codes.json ...
... OK


In [3]:
%autoreload 2
demoji.download_codes()

... OK (Got response in 0.18 seconds)
Writing emoji data to C:\Users\Administrator\.demoji\codes.json ...
... OK


In [4]:
def disp_col(list_):
    
    # import -> from IPython.utils.text import columnize
    l = list(map(lambda x:repr(x)+',', list_))
    print(columnize(l, displaywidth=120))

In [5]:
disp_col(dir(scp))

'SpotifyClientCredentials',  '__loader__',        'get_artist_name',       'pd',                   'user_plid_pair',
'__builtins__',              '__name__',          'get_public_playlists',  're',                 
'__cached__',                '__package__',       'get_tracks',            'spotipy',            
'__doc__',                   '__spec__',          'get_tracks_df',         'spotipy_client_cred',
'__file__',                  'filterby_keyword',  'json_normalize',        'uri_to_id',          



In [6]:
disp_col(dir(sc))

'Path',          '__name__',        'filtersort_playlists',   'get_tracks',           'spotipy',               'util',
'__builtins__',  '__package__',     'get_folder_analysis',    'get_tracks_analysis',  'spotipy_userauth',    
'__cached__',    '__spec__',        'get_pl_details',         'json_normalize',       'spotipy_userauth2',   
'__doc__',       'convert_time',    'get_playlist_analysis',  'pd',                   'track_anlaysis_to_df',
'__file__',      'create_dataset',  'get_playlists',          'playlists_id_url',     'track_genre',         
'__loader__',    'demoji',          'get_segments',           're',                   'tracks_analysis',     



In [7]:
load_dotenv()
SPOTIPY_CLIENT_ID = os.getenv('CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REDIRECT_URI = os.getenv('REDIRECT_URI')
USERNAME = os.getenv('SPOTIFY_USERNAME')
SCOPE = 'playlist-read-private'

In [8]:
sp = scp.spotipy_client_cred(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)

In [9]:
def timer(start_time=None):
    if start_time is not None:
        return (datetime.now() - start_time).total_seconds()
    else:
        start_time = datetime.now()
        return start_time

In [10]:
path_  = Path.cwd().joinpath('Dataset').joinpath('musiclabels_tracks.parquet')
tracks_df = pd.read_parquet(path_)
tracks_df.tail()

,name,id,artists_name,user
18381,Volume One - Free State vs Dirt Devils Remix,03NSRR3kgp4oOKqGtTEjOZ,Anjunabeats,Anjunabeats
18382,Volume One - Anjuna Deep Mix,4qKcK7oKhB2oMF1Ny2T4Rr,Anjunabeats,Anjunabeats
18383,Volume One - Original Mix,1ZfyvweqqR3csr8K4ACjt4,Anjunabeats,Anjunabeats
18384,Volume One - Tease Dub,4GYyq7cPaTjyynNWpPkC9P,Anjunabeats,Anjunabeats
18385,Won't Sleep Tonight - Moody Dub Mix,3G2fWbQSseRglXGRhC99Jm,Super8 & Tab,Anjunabeats


In [11]:
tracks_df = tracks_df.replace('vcdikomxn7xuhp0ci0pn31pn9', 'ChillBeatsMusic')

In [12]:
users = list(tracks_df['user'].unique())
print(users)

['ChillBeatsMusic', 'anjunadeep', 'spinninrecordsofficial', 'enhanced_music', 'emergentmusic', 'armadamusicofficial', 'spotify', 'silkmusic', 'Anjunabeats']


In [13]:
tracksdf_byuser = [tracks_df[tracks_df['user'] == u] for u in users]

In [14]:
tracksdf_chunks = []
n = 100 # chunk size
for df in tracksdf_byuser:
    
    list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]
    tracksdf_chunks.append(list_df)
    
print(len(tracksdf_byuser))
for u,l in zip(users, tracksdf_chunks):
    
    print('user : {} , # of dfs : {}'.format(u, len(l)))

9
user : ChillBeatsMusic , # of dfs : 3
user : anjunadeep , # of dfs : 19
user : spinninrecordsofficial , # of dfs : 33
user : enhanced_music , # of dfs : 2
user : emergentmusic , # of dfs : 3
user : armadamusicofficial , # of dfs : 58
user : spotify , # of dfs : 45
user : silkmusic , # of dfs : 5
user : Anjunabeats , # of dfs : 19


In [15]:
path = Path.cwd().joinpath('Dataset')
path

WindowsPath('C:/Users/Administrator/Desktop/CapstoneJup/Dataset')

In [16]:
def get_df_analysis(spotipyUserAuth, tracks_df, segments=True, min_conf=0.5,
                          min_dur=0.25, tempo=True, sections=False, beats=False, bars=False):
    '''
    spotipyUserAuth : Spotipy auth object.
    playlist_id : playlist id
    segments and tempo: Default True. False if not needed
    min_conf: minimum confidence to include a segment (range 0-1)
    min_dur : minimum duration/length in secs to include a segment
    sections/beats/bars: Default False. True if needs to be returned

    Returns : a dict with key/value pairs for all tracks in the playlist
                Keys: name of track
                Value: list containing tempo and segment dataframe of the track
                       (and sections/beats/bars if asked)
    '''

    tracks_name = list(tracks_df['name'])
    tracks_id = list(tracks_df['id'])
    artists_name = list(tracks_df['artists_name'])
    # track_analysis returns a list of dictionary
    tracks_analysis = sc.get_tracks_analysis(spotipyUserAuth, tracks_id)
    df_analysis = {}

    for name_, artists_name_, track_analysis in zip(tracks_name, artists_name, tracks_analysis):

        # remove any special characters from name (they may cause issues in filenaming)
        name_ = re.sub(r'[*|><:"?/]|\\', "", name_)
        name_ = demoji.replace(name_)
        artists_name_ = re.sub(r'[*|><:"?/]|\\', "", artists_name_)
        name_ = name_ + '-' + artists_name_[:3]
        df_analysis[name_] = sc.get_segments(track_analysis, segments=segments,
                                                min_conf=min_conf, min_dur=min_dur, tempo=tempo,
                                                sections=sections, beats=beats, bars=bars)
    return df_analysis

In [17]:
def user_analysis(sp, user, df, save=True, path=None, fn=0):    
    
    if save:
        # Create user folder
        path_ = path.joinpath('{}'.format(user))
        path_.mkdir(exist_ok=True)
        # create sub folders for every chunk or n number of tracks in the user folder
        p = path_.joinpath('{}_{}'.format(u, fn))
        p.mkdir(exist_ok=True)

    # list of dataframe names in output
    df_names = ['tempo', 'segments', 'sections', 'beats', 'bars']
        
    df_analysis = get_df_analysis(sp, df)
        
    for track, a in df_analysis.items():
        
        for k in range(len(a)):

            a[k].to_parquet(p.joinpath('{}_{}.parquet'.format(track, df_names[k])), engine='pyarrow')

In [ ]:
j = 6
k = 7

for u, user_dfs in zip(users[j:k], tracksdf_chunks[j:k]): 
    
    for i in range(26, 29):
        
        user_analysis(sp, user=u, df=user_dfs[i], path=path, fn=i)
        

In [27]:
spoti = tracks_df[tracks_df['user']=='spotify']

In [28]:
spoti

,name,id,artists_name,user
11630,Sing To Me,17KGTOb8fkxlyTnB5eFnsh,"MISSIO, Death Stranding: Timefall",spotify
11631,Strange Clouds,7o9fFaoToMlmcTftxZmgMs,ufo ufo,spotify
11632,The Kid I Used to Know,4poqovyJtmlKMDnSz6QwsE,Arrested Youth,spotify
11633,Our House (The Mess We Made),5bl2pNsPhMT37RkDUa57Ab,You Me At Six,spotify
11634,Dead Weight,7hM9ShrZMCV7xYOQwqBnNb,PVRIS,spotify
...,...,...,...,...
16034,Kafka,6QlAW39LcyCNTxqsYjRBki,Ercos Blanka,spotify
16035,Branches,6q5QgqXvW1lgIBSdSihcVN,Nora En Pure,spotify
16036,Whispers,0V8okvZJmTFqnbsW9L1v1N,Valante,spotify
16037,Setting Sail - Chill Mix,3Am7bRVPs1DPbTkYQb9giH,Eastern Odyssey,spotify


In [35]:
spoti[spoti['name']=='AEIOU']

,name,id,artists_name,user
14584,AEIOU,7KWkq3RScihPRrbDn8sqyg,"Joell, Billy Got Waves",spotify


In [36]:
spoti.loc[14210:14584]

,name,id,artists_name,user
14210,Wasted Sunsets,1IyTlom5MjqWmQe4mnc6TC,Deep Purple,spotify
14211,Bloodsucker - 1995 Remaster,4f2Mreghv4pgUIWVDJG0X7,Deep Purple,spotify
14212,Lady Double Dealer,4BJAeOVsTFqZOr9Ih3YT8R,Deep Purple,spotify
14213,All I Got Is You,1mEWmFEjFh1rGqZ8QkkJ0j,Deep Purple,spotify
14214,Lady Luck - 2010 Digital Remaster,0GTZdFCyqZ0gTItkBaiE3i,Deep Purple,spotify
...,...,...,...,...
14580,Be Mindful,3MozP0utcd8PxTQ4BIcbmt,"Yeti, Mase",spotify
14581,Big Bad,30avbfWF8K8ZodTwNHTLog,Jaywop,spotify
14582,Toyota Ae86,7pPSVe2XBmjkhFnvLS9M2F,andy polk,spotify
14583,Need Sum,3qF7hj0Ntqew5mWdEG92cm,CP,spotify


In [100]:
start_time = timer(None)
i = 14435+80

df_analy = get_df_analysis(sp, spoti.loc[i:i+5])

timer(start_time)

3.974199

In [82]:
path_ = path.joinpath('{}'.format('spotify'))
path_.mkdir(exist_ok=True)
        # create sub folders for every chunk or n number of tracks in the user folder
p = path_.joinpath('{}_{}'.format('spotify', 28))
p.mkdir(exist_ok=True)

In [101]:
df_names = ['tempo', 'segments', 'sections', 'beats', 'bars']
for track, a in df_analy.items():
        
        for k in range(len(a)):

            a[k].to_parquet(p.joinpath('{}_{}.parquet'.format(track, df_names[k])), engine='pyarrow')